# Model training with YOLOv5

## Prepare YOLO

This step is pretty easy, we only have to download YOLOv5 from Ultralytics and install the necessary dependencies

### Download YOLOv5

In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16074, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 16074 (delta 5), reused 9 (delta 1), pack-reused 16056
Receiving objects: 100% (16074/16074), 14.69 MiB | 21.31 MiB/s, done.
Resolving deltas: 100% (11033/11033), done.


### We must slightly edit the requirements to have a container-compatible version of openCV

In [2]:
!sed -i 's/opencv-python/opencv-python-headless/' yolov5/requirements.txt

### Then install the requirements

In [2]:
!pip install -qr yolov5/requirements.txt


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


### Unfortunately, some subdependencies may have messed up with OpenCV. So let's make sure again we have the right version again...

In [3]:
!pip uninstall -qy opencv-python
!pip uninstall -qy opencv-python-headless
!pip install -q opencv-python-headless


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


## Training

### Create the configuration file

There is a `configuration.yaml` file already present in the folder. Verify that it has the right number and names of the classes you want to use and that you downloaded.

### Freeze the YOLOv5 Backbone
The backbone means the layers that extract input image features. We will freeze the backbone so the weights in the backbone layers will not change during YOLOv5 transfer learning. We will then only train the last layers (the head layers).

In this example, we will use the smallest available base model, yolov5n. If we open the file `yolov5/models/yolov5n.yaml` we can see the following structure:

In the **backbone** section we can see that there are 10 layers. So that's the number of layers we want to freeze in our training!

### Some training parameters (hyperparameters) we are going to use

#### The base model
weights = yolov5n.pt

This is the smallest available model, to save processing time in this example. Adjust for better results.
Other models are available here: https://github.com/ultralytics/yolov5#pretrained-checkpoints

####  The number of training iterations
epochs = 50

This is voluntarily low to save processing time for this example. Adjust for better results (>150).

#### The number of images analyzed in a single pass
batch = 256

You may have to adjust this depending on the memory available on your GPU. The higher (in a power of 2) the better, until you run out of memory...

#### Number of layers to freeze
freeze = 10

As per above, we want to freeze 10 layers

### We can now launch the training!

NOTE: PyTorch first caches images to speed up the process. If you have enough memory and shared memory that is not an issue. However, this may not be always the case, especially with large datasets. Therefore the cache is forced to disk here. But of course this is a parameter you can change.

In [12]:
!python yolov5/train.py --data configuration.yaml --weights yolov5s.pt --epochs 50 --batch 16 --freeze 10 --cache disk

train: weights=yolov5s.pt, cfg=, data=configuration.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=disk, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-240-g84ec8b5 Python-3.9.16 torch-1.13.1+cu117 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, de

In [ ]:
!python yolov5/train.py --resume yolov5/runs/train/exp13/weights/last.pt

train: weights=yolov5/yolov5s.pt, cfg=, data=yolov5/data/coco128.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=yolov5/runs/train/exp13/weights/last.pt, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-240-g84ec8b5 Python-3.9.16 torch-1.13.1+cu117 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, c

The model has been saved under `yolov5/runs/train/exp/weights/best.pt`.

### Test the model

We can now simply test the model against an image: `test/img_test.jpg`
The image will be analyzed, enriched with bounding boxes, and the result will be saved under `yolov5/runs/detect/exp`

In [5]:
!python yolov5/detect.py --weights yolov5/runs/train/exp12/weights/best.pt --img 640 --conf-thres 0.2 --source test

detect: weights=['yolov5/runs/train/exp12/weights/best.pt'], source=test, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-240-g84ec8b5 Python-3.9.16 torch-1.13.1+cu117 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/4 /opt/app-root/src/yolov5-transfer-learning/test/adrien-redhat.jpg: 640x480 1 redhat, 139.3ms
image 2/4 /opt/app-root/src/yolov5-transfer-learning/test/image-1.jpg: 640x640 3 redhats, 127.0ms
image 3/4 /opt/app-root/src/yolov5-transfer-learning/test/image-17.jpg: 640x640 2 redhats, 159.3ms
image 4/4 /opt/app-root/src/yolov5-transfer-lea